In [1217]:
import math
import random
import os
from pprint import pprint

#### Configuration (Modify here only)

In [1218]:
filename = "D:/VLSI/Capstone/tb/v2/top_tb.sv"
root_path = r"D:/VLSI/Capstone"

# filename = r"\\wsl$\Ubuntu-22.04\home\hwanzar\capstone241\graph-attention-network-fpga\tb\top_tb.sv"
# root_path = r"\\wsl$\Ubuntu-22.04\home\hwanzar\capstone241\graph-attention-network-fpga"

In [1219]:
params = {
  "H_NUM_OF_ROWS" : 100,
  "H_NUM_OF_COLS" : 8,

  "W_NUM_OF_ROWS" : 8,
  "W_NUM_OF_COLS" : 16,

  "NUM_OF_NODES"  : 6,
}

#### 1. Random Inputs

In [1220]:
params.update({
  "COL_IDX_DEPTH"   : 0,                                # số lượng col_idx tổng cộng
  "VALUE_DEPTH"     : 0,                                # số lượng value tổng cộng
  "NODE_INFO_DEPTH" : 0,                                # số lượng node tổng cộng
  "WEIGHT_DEPTH"    : 0,                                # = W_ROWS * W_COLS
  "WH_DEPTH"        : 0,                                # = số lượng node tổng cộng
  "A_DEPTH"         : params["W_NUM_OF_COLS"] * 2,      # = W_COLS * 2
})

In [1221]:
def random_matrix(rows, cols, sparsity_percent):
	total_elements = rows * cols
	num_zeros = int(total_elements * sparsity_percent / 100)
	num_non_zeros = total_elements - num_zeros
	matrix_elements = [0] * num_zeros + [random.randint(1, 5) for _ in range(num_non_zeros)]
	random.shuffle(matrix_elements)
	sparse_matrix = []
	for i in range(rows):
		sparse_matrix.append(matrix_elements[i * cols:(i + 1) * cols])
	for row in sparse_matrix:
		if all(value == 0 for value in row):
			row[random.randint(0, cols - 1)] = random.randint(1, 9)
	return sparse_matrix

##### Weight


In [1222]:
weight = random_matrix(params["W_NUM_OF_ROWS"], params["W_NUM_OF_COLS"], 0)
# weight = [[1] * params["W_NUM_OF_COLS"] for _ in range(params["W_NUM_OF_ROWS"])]

##### Weight Vector a

In [1223]:
a = [1 if i < params["A_DEPTH"] / 2 else 2 for i in range(params["A_DEPTH"])]
# a = random_matrix(1, params["A_DEPTH"], 0)[0]

##### SubGraph

In [1224]:
subGraph = []
sub_row = random.randint(2, params["NUM_OF_NODES"])
sub_cnt = 0
source_node_list = []
while sub_cnt < params["H_NUM_OF_ROWS"]:
  sub_row = random.randint(2, params["NUM_OF_NODES"])
  sub_cnt += sub_row
  if sub_cnt >= params["H_NUM_OF_ROWS"]:
    sub_row = sub_row - (sub_cnt - params["H_NUM_OF_ROWS"])
    if sub_row == 1:
      sub_row = 2
      for item in subGraph:
        if len(item) >= 3:
          item.pop(-1)
          break
  subG = random_matrix(sub_row, params["H_NUM_OF_COLS"], 50)

  subGraph.append(subG)

print("Num of subGraph = ", len(subGraph))

Num of subGraph =  29


##### Feature

In [1225]:
feature = []
for i in range(len(subGraph)):
  feature += subGraph[i]

print("H_NUM_OF_ROWS = ", len(feature))

H_NUM_OF_ROWS =  100


#### 2. Calculation

##### SPMM

In [1239]:
wh = [[0 for _ in range(len(weight[0]))] for _ in range(len(feature))]
for i in range(len(feature)):
  for j in range(len(weight[0])):
    for k in range(len(weight)):
      wh[i][j] += feature[i][k] * weight[k][j]

def matrix_multiplication(subgraph, weight_matrix):
  rows = len(subgraph)
  cols = len(weight_matrix[0])
  common_dim = len(weight_matrix)

  result_matrix = [[0 for _ in range(cols)] for _ in range(rows)]

  for i in range(rows):
    for j in range(cols):
      for k in range(common_dim):
        result_matrix[i][j] += subgraph[i][k] * weight_matrix[k][j]
  return result_matrix

WH = []
for i in range(len(subGraph)):
	wh_temp = matrix_multiplication(subGraph[i], weight)
	WH.append(wh_temp)

[[20, 25, 19, 24, 16, 19, 28, 18, 24, 17, 19, 24, 25, 18, 13, 13],
 [45, 23, 26, 29, 28, 35, 25, 28, 29, 16, 25, 37, 26, 23, 14, 31]]


##### DMVM

In [1248]:
midpoint = len(a) // 2
a_1 = a[:midpoint]
a_2 = a[midpoint:]

dmvm = []
coef = []

# for index, row in enumerate(wh):
#   sum = 0
#   for i in range(len(row)):
#     if index % params["NUM_OF_NODES"] == 0:
#       sum += row[i] * a_1[i]
#     else:
#       sum += row[i] * a_2[i]
#   dmvm.append(sum)
# for i in range(len(dmvm)):
#   coef.append(dmvm[i] + dmvm[int(i / params["NUM_OF_NODES"]) * params["NUM_OF_NODES"]])

for wh in WH:
  for row in wh:
    for i in range(len(row)):
      sum = 0
      if (i == 0):
        sum += row[i] * a_1[i]
      else:
        sum += row[i] * a_2[i]
      dmvm.append(sum)

# print(dmvm)

[20, 50, 38, 48, 32, 38, 56, 36, 48, 34, 38, 48, 50, 36, 26, 26, 45, 46, 52, 58, 56, 70, 50, 56, 58, 32, 50, 74, 52, 46, 28, 62, 73, 90, 80, 80, 116, 124, 100, 84, 108, 106, 116, 104, 142, 104, 82, 128, 40, 80, 68, 90, 52, 62, 94, 66, 76, 44, 64, 90, 74, 62, 42, 46, 27, 62, 42, 34, 58, 62, 46, 42, 62, 50, 42, 50, 72, 50, 34, 52, 64, 88, 88, 92, 86, 94, 60, 82, 74, 62, 70, 116, 90, 94, 54, 94, 86, 94, 102, 150, 102, 148, 136, 106, 138, 146, 140, 144, 136, 84, 82, 126, 30, 20, 24, 50, 26, 46, 52, 38, 38, 12, 42, 60, 22, 16, 16, 28, 34, 84, 36, 72, 58, 72, 58, 60, 86, 56, 46, 78, 60, 46, 50, 58, 35, 54, 36, 46, 50, 42, 18, 44, 44, 30, 30, 50, 42, 54, 42, 64, 82, 102, 112, 152, 84, 92, 72, 100, 90, 102, 92, 132, 96, 110, 82, 122, 44, 100, 92, 88, 72, 74, 90, 70, 80, 74, 72, 94, 110, 92, 52, 64, 51, 62, 46, 92, 60, 96, 88, 72, 92, 56, 74, 104, 60, 36, 42, 64, 25, 20, 32, 46, 20, 32, 26, 30, 24, 22, 28, 46, 20, 22, 14, 28, 42, 46, 52, 72, 46, 52, 70, 50, 50, 48, 68, 70, 64, 56, 44, 56, 25, 5

##### Softmax

In [ ]:
# TODO:

##### Aggregator

In [ ]:
# TODO:

#### 3. GCSR structure

In [1228]:
source_node_list = []
num_nodes_subgraph = []
node_info = []
col_idx = []
value = []
non_zero_ele = []

_row_bits = math.log2(params["H_NUM_OF_COLS"])
_numnode_bits = math.log2(params["NUM_OF_NODES"])

row_length_bits = int(_row_bits) + 1 if _row_bits.is_integer() else math.ceil(_row_bits)
num_of_nodes_bits = int(_numnode_bits) + 1 if _numnode_bits.is_integer() else math.ceil(_numnode_bits)


for i in range(len(subGraph)):
  source_node = subGraph[i][0]
  num_node = len(subGraph[i])
  source_node_list.append(source_node)
  num_nodes_subgraph.append(num_node)
  node_info_list = []
  for j in range(len(subGraph[i])):
    if j == 0:
      flag = '1'
    else:
      flag = '0'
    non_zero_values = [ele for k, ele in enumerate(subGraph[i][j]) if ele != 0]
    # print("Non Zero Values = ", non_zero_values)
    row_length = len(non_zero_values)
    node_info_bin = (f"{row_length:0{row_length_bits}b}"f"{num_node:0{num_of_nodes_bits}b}" f"{flag}")
    node_info_list.append(node_info_bin)

  node_info.append(node_info_list)

def decode_node_info(node_info_string):
    # Extract each part from the string
    flag_bits = 1
    row_length = int(node_info_string[:row_length_bits], 2)
    num_of_nodes = int(node_info_string[row_length_bits:row_length_bits+num_of_nodes_bits], 2)

    flag = node_info_string[-flag_bits:]  # '1' for source node, '0' for neighbor node

    # Return a dictionary for better readability
    return {
        'row_length': row_length,
        'num_of_nodes': num_of_nodes,
        'is_source_node': flag == '1'
    }

# Example usage
decoded_node_info = []
for nd_info in node_info:
    decoded_info = [decode_node_info(info) for info in nd_info]
    decoded_node_info.append(decoded_info)
# pprint(decoded_node_info)

In [1229]:
def extract_from_raw(raw_list):
    col_idx = []
    value = []
    node_info = []
    for row in raw_list:
        non_zero_elements = [(j, ele) for j, ele in enumerate(row) if ele != 0]
        col_idx.extend([idx for idx, _ in non_zero_elements])
        value.extend([val for _, val in non_zero_elements])
        row_len = len(non_zero_elements)

    return col_idx, value

#### 4. SubGraph Dictionary

In [1237]:
GRAPH = {}
DMVM = []
COEF_I = []
idx = 0
for subgraph in subGraph:
  dmvm_temp = []
  coef_temp = []
  for i in range(len(subgraph)):
    dmvm_temp.append(dmvm[idx])
    coef_temp.append(coef[idx])
    idx += 1
  DMVM.append(dmvm_temp)
  COEF_I.append(coef_temp)
for i in range(len(subGraph)):
  col_idx_, value_ = extract_from_raw(subGraph[i])
  GRAPH[i] = {"raw" : subGraph[i], "col_idx" : col_idx_, "value" : value_, "node_info" : node_info[i], "WH": WH[i],  "DMVM": DMVM[i], "Coef": COEF_I[i]}

colors = {
  "raw": "\033[91m",       # Red
  "col_idx": "\033[92m",   # Green
  "value": "\033[93m",     # Yellow
  "node_info": "\033[94m", # Blue
  "WH": "\033[95m",        # Magenta
  "DMVM": "\033[96m",      # Cyan
  "Coef": "\033[97m",      # White
  "reset": "\033[0m"       # Reset color
}

for subgraph in GRAPH.values():
  print("-" * 50)
  for i in range(len(subgraph["raw"])):
    if i == 0:
      print(f"{colors['raw']}raw\t\t{subgraph['raw'][i]}{colors['reset']}")
    else:
      print(f"{colors['raw']}\t\t{subgraph['raw'][i]}{colors['reset']}")

  print(f"{colors['col_idx']}col_idx\t\t{subgraph['col_idx']}{colors['reset']}")
  print(f"{colors['value']}value\t\t{subgraph['value']}{colors['reset']}")
  print(f"{colors['node_info']}node_info\t{subgraph['node_info']}{colors['reset']}")

  for i in range(len(subgraph["WH"])):
    if i == 0:
      print(f"{colors['WH']}SPMM\t\t{subgraph['WH'][i]}{colors['reset']}")
    else:
      print(f"{colors['WH']}\t\t{subgraph['WH'][i]}{colors['reset']}")

  print(f"{colors['DMVM']}DMVM\t\t{subgraph['DMVM']}{colors['reset']}")
  print(f"{colors['Coef']}Coef\t\t{subgraph['Coef']}{colors['reset']}")
  print("-" * 50)

--------------------------------------------------
raw		[1, 0, 3, 2, 0, 1, 0, 0]
		[1, 1, 0, 4, 0, 0, 3, 0]
col_idx		[0, 2, 3, 5, 0, 1, 3, 6]
value		[1, 3, 2, 1, 1, 1, 4, 3]
node_info	['01000101', '01000100']
SPMM		[20, 25, 19, 24, 16, 19, 28, 18, 24, 17, 19, 24, 25, 18, 13, 13]
		[45, 23, 26, 29, 28, 35, 25, 28, 29, 16, 25, 37, 26, 23, 14, 31]
DMVM		[322, 880]
Coef		[644, 1202]
--------------------------------------------------
--------------------------------------------------
raw		[0, 0, 0, 3, 0, 4, 5, 5]
		[2, 0, 5, 5, 0, 0, 0, 0]
		[1, 0, 1, 0, 0, 3, 3, 0]
		[1, 0, 0, 2, 5, 2, 4, 0]
		[0, 4, 1, 5, 2, 5, 0, 4]
		[1, 0, 0, 5, 0, 0, 0, 0]
col_idx		[3, 5, 6, 7, 0, 2, 3, 0, 2, 5, 6, 0, 3, 4, 5, 6, 1, 2, 3, 4, 5, 7, 0, 3]
value		[3, 4, 5, 5, 2, 5, 5, 1, 1, 3, 3, 1, 2, 5, 2, 4, 4, 1, 5, 2, 5, 4, 1, 5]
node_info	['01001101', '00111100', '01001100', '01011100', '01101100', '00101100']
SPMM		[73, 45, 40, 40, 58, 62, 50, 42, 54, 53, 58, 52, 71, 52, 41, 64]
		[40, 40, 34, 45, 26, 31, 47, 33, 

#### 5. Update testbench

##### Inputs

In [1231]:
file_path_input = os.path.join(root_path, "tb/inputs")

In [1232]:
wrt_col_idx = []
wrt_value = []
wrt_node_info = []

for key, value in GRAPH.items():
  wrt_col_idx.append(value["col_idx"])
  wrt_value.append(value["value"])
  wrt_node_info.append(value["node_info"])

In [1233]:
def write_list_to_file(file_path, data_list):
  with open(file_path, 'w') as file:
    print(f" open {file_path}" )
    for item in data_list:
      for it in item:
        file.write(f"{it}\n")
def write_list_to_file2(file_path, data_list):
  with open(file_path, 'w') as file:
    print(f" open {file_path}" )
    for item in data_list:
      file.write(f"{item}\n")

write_list_to_file(file_path_input + "/col_idx.txt", wrt_col_idx)
write_list_to_file(file_path_input + "/value.txt", wrt_value)
write_list_to_file(file_path_input + "/node_info.txt", wrt_node_info)
write_list_to_file(file_path_input + "/weight.txt", weight)
write_list_to_file2(file_path_input + "/a.txt", a)

 open D:/VLSI/Capstone\tb/inputs/col_idx.txt
 open D:/VLSI/Capstone\tb/inputs/value.txt
 open D:/VLSI/Capstone\tb/inputs/node_info.txt
 open D:/VLSI/Capstone\tb/inputs/weight.txt
 open D:/VLSI/Capstone\tb/inputs/a.txt


##### Parameters

In [1234]:
_col_idx = [len(item) for item in wrt_col_idx]
len_col_idx = 0
for col in _col_idx:
  len_col_idx += col

_value = [len(item) for item in wrt_value]
len_value = 0
for val in _value:
  len_value += val

_node_info = [len(item) for item in wrt_node_info]
len_node_info = 0
for node in _node_info:
  len_node_info += node

params["COL_IDX_DEPTH"] = len_col_idx
params["VALUE_DEPTH"] = len_value
params["NODE_INFO_DEPTH"] = len_node_info
params["WEIGHT_DEPTH"] = params["W_NUM_OF_ROWS"] * params["W_NUM_OF_COLS"]
params["WH_DEPTH"] = len_node_info
params["A_DEPTH"] = params["W_NUM_OF_COLS"] * 2
pprint(params)

{'A_DEPTH': 32,
 'COL_IDX_DEPTH': 400,
 'H_NUM_OF_COLS': 8,
 'H_NUM_OF_ROWS': 100,
 'NODE_INFO_DEPTH': 100,
 'NUM_OF_NODES': 6,
 'VALUE_DEPTH': 400,
 'WEIGHT_DEPTH': 128,
 'WH_DEPTH': 100,
 'W_NUM_OF_COLS': 16,
 'W_NUM_OF_ROWS': 8}


In [1235]:
def update_parameters(filename, parameters):
  with open(filename, 'r') as file:
    file_data = file.readlines()

  for i, line in enumerate(file_data):
    for param, new_value in parameters.items():
      found_param = "parameter " + param
      # print(param)
      if found_param in line:
        file_data[i] = f"  parameter {param}\t\t\t= {new_value},\n"

  with open(filename, 'w') as file:
      file.writelines(file_data)

update_parameters(filename, params)

##### Root Path

In [1236]:
def update_root_path(filename, root_path):
  with open(filename, 'r') as file:
    file_data = file.readlines()

  for i, line in enumerate(file_data):
    if "localparam string ROOT_PATH" in line:
      file_data[i] = f"\tlocalparam string ROOT_PATH = \"{root_path}\";\n"

  with open(filename, 'w') as file:
      file.writelines(file_data)

update_root_path(filename, root_path)